<a href="https://colab.research.google.com/github/PaulToronto/Stanford-Andrew-Ng-Machine-Learning-Specialization/blob/main/Overflow_safe_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overflow safe algorithm

## Imports

In [1]:
import numpy as np

## Functions

In [2]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Parameters
    ----------
    z : array_like
        A scalar or numpy array of any size.

    Returns
    -------
     g : array_like
         sigmoid(z)
    """
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

In [3]:
def log_1pexp(x, maximum=20):
    ''' approximate log(1+exp^x)
        https://stats.stackexchange.com/questions/475589/numerical-computation-of-cross-entropy-in-practice
    Args:
    x   : (ndarray Shape (n,1) or (n,)  input
    out : (ndarray Shape matches x      output ~= np.log(1+exp(x))
    '''

    out  = np.zeros_like(x,dtype=float)
    i    = x <= maximum
    ni   = np.logical_not(i)

    out[i]  = np.log(1 + np.exp(x[i]))
    out[ni] = x[ni]
    return out

## Compare algorithms

### Data

In [4]:
X = np.random.rand(6, 2)
y = np.array([0, 0, 0, 1, 1, 1]).reshape(-1, 1)
X, y

(array([[0.53220283, 0.40790202],
        [0.88045336, 0.21175131],
        [0.90223389, 0.1655283 ],
        [0.80432846, 0.90127959],
        [0.83611376, 0.79212886],
        [0.01755092, 0.66547262]]),
 array([[0],
        [0],
        [0],
        [1],
        [1],
        [1]]))

In [5]:
w = np.random.rand(2, 1)
b = np.random.rand(1)[0]
w, b

(array([[0.50594551],
        [0.40416586]]),
 0.5507352038138403)

### Safe vs. Unsafe

In [6]:
def safe_cost(X, y, w, b):
    m = X.shape[0]
    z = X @ w + b
    cost = -(y * z) + log_1pexp(z)
    cost = np.sum(cost) / m
    return cost

In [7]:
def unsafe_cost(X, y, w, b):
    m = X.shape[0]
    f = sigmoid(X @ w + b)
    cost = (1 / m) * (np.dot(-y.T, np.log(f)) - np.dot((1-y).T, np.log(1-f)))
    cost = cost[0,0]
    return cost

In [8]:
safe_cost(X, y, w, b) - unsafe_cost(X, y, w, b)

2.220446049250313e-16

In [9]:
collect_all = []
for i in range(100):
    X = np.random.rand(6, 2)
    y = np.random.randint(0, 2, 6).reshape(-1, 1)
    w = np.random.rand(2, 1)
    b = np.random.rand(1)[0]
    difference = safe_cost(X, y, w, b) - unsafe_cost(X, y, w, b)
    collect_all.append(difference)

In [10]:
collect_all

[1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 0.0,
 0.0,
 5.551115123125783e-17,
 2.220446049250313e-16,
 0.0,
 2.220446049250313e-16,
 0.0,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 -2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 -1.1102230246251565e-16,
 0.0,
 -1.1102230246251565e-16,
 5.551115123125783e-17,
 0.0,
 1.1102230246251565e-16,
 -2.220446049250313e-16,
 0.0,
 0.0,
 -5.551115123125783e-17,
 0.0,
 0.0,
 -2.220446049250313e-16,
 0.0,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 5.551115123125783e-17,
 3.3306690738754696e-16,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 -1.1102230246251565e-16,
 0.0,
 -2.220446049250313e-16,
 -2.220446049250313e-16,
 4.440892098500626e-16,
 -1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 3.3306690738754696e-16,
 2.220446049250313e-16,
 2.220446049250313e-16,
 -1.1102230246251565e-16,
 4.440892098500626e-16,
 0.0,
 0.0,
 0.0,
 1.1102230246251565

In [11]:
sum(collect_all)

3.7192471324942744e-15